# Activity Sampling

Parameters:

In [ ]:
LOG_FILE = "../desktop-app/data/activity-log.csv"

Read activity log and compute activities ...

In [ ]:
import collections
import csv
import datetime
import math

# TODO add working hours
Activity = collections.namedtuple(
    "Activity",
    ["start", "finish", "client", "project", "task", "cycle_time"]
)


def project_activities(log_file=LOG_FILE):
    activities = {}
    with open(log_file, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = (row['Client'], row['Project'], row['Task'])
            if key not in activities:
                date = datetime.datetime.fromisoformat(row['Timestamp']).date()
                activity = Activity(
                    start=date,
                    finish=date,
                    client=row['Client'],
                    project=row['Project'],
                    task=row['Task'],
                    cycle_time=datetime.timedelta(days=1)
                )
                activities[key] = activity
            else:
                activity = activities[key]
                date = datetime.datetime.fromisoformat(row['Timestamp']).date()
                start_updated = min(activity.start, date)
                finish_updated = max(activity.finish, date)
                activity = Activity(
                    start=start_updated,
                    finish=finish_updated,
                    client=row['Client'],
                    project=row['Project'],
                    task=activity.task,
                    cycle_time=finish_updated - start_updated + datetime.timedelta(
                        days=1)
                )
                activities[key] = activity
    return [activity for activity in activities.values()]


activities = project_activities()

## List of Activities

In [ ]:
def print_activities(activities):
    activities = sorted(activities, key=lambda x: (x.task, x.project, x.client))
    headers = ["Start", "Finish", "Task", "Project", "Client", "Cycle Time"]
    print(
        f"{headers[0]:<10}  {headers[1]:<10}  {headers[2]:<30}  {headers[3]:<15}  {headers[4]:<15}  {headers[5]:<10}")
    print("-" * 100)
    for activity in activities:
        task = activity.task[:30]
        project = activity.project[:15]
        client = activity.client[:15]
        start = str(activity.start)
        finish = str(activity.finish)
        cycle_time_days = math.ceil(
            activity.cycle_time.total_seconds() / 3600 / 24)
        print(
            f"{start:<10}  {finish:<10}  {task:<30}  {project:<15}  {client:<15}  {cycle_time_days:>10}")


print_activities(activities)

## Cycle Times

In [ ]:
def print_cycle_times(activities):
    cycle_time_counts = collections.Counter()
    for activity in activities:
        cycle_time_days = math.ceil(
            activity.cycle_time.total_seconds() / 3600 / 24)
        cycle_time_counts[cycle_time_days] += 1
    sorted_cycle_times = sorted(cycle_time_counts.items())
    headers = ["Cycle Time", "Frequency", "Percentage", "Cumulative"]
    print(
        f"{headers[0]:<10}  {headers[1]:<9}  {headers[2]:<10}  {headers[3]:<10}")
    print("-" * 45)
    total = cycle_time_counts.total()
    cumulative = 0
    for cycle_time, frequency in sorted_cycle_times:
        percentage = frequency / total * 100
        cumulative += percentage
        print(
            f"{cycle_time:>10}  {frequency:>9}  {round(percentage):>9}%  {round(cumulative):>9}%")


print_cycle_times(activities)